# Introduction - A Chatbot that uses RAG with Weaviate

Implements a simple workflow of constant retrieve then generate chains. Using `semantic kernel` to help with prompt engineering and orchestrating calls to LLMs and Weaviate as knowledgebase from which to retreive semantically relevant context.

This chatbot not only answers using relevant retreived content but also cites sources and relevance.

## Setup

In [ ]:
#!pip install semantic-kernel==0.3.0.dev0
#!pip install weaviate-client
#!pip install python-dotenv

## OS-specific notes:
* if you run into SSL errors when connecting to OpenAI on macOS, see this issue for a [potential solution](https://github.com/microsoft/semantic-kernel/issues/627#issuecomment-1580912248)
* on Windows, you may need to run Docker Desktop as administrator

In [1]:
from typing import Tuple
import os
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAITextCompletion,
    OpenAITextEmbedding,
)

In [2]:
from dotenv import load_dotenv

In [3]:
from semantic_kernel.connectors.memory.weaviate import weaviate_memory_store
from dotenv import load_dotenv

load_dotenv(verbose=True, override=True)

# Using Docker
config = weaviate_memory_store.WeaviateConfig(url="http://localhost:8080")

store = weaviate_memory_store.WeaviateMemoryStore(config=config)
store.client.schema.delete_all()

Then, we register the memory store to the kernel:

In [4]:
kernel = sk.Kernel()

api_key, org_id = sk.openai_settings_from_dot_env()

kernel.add_text_completion_service(
    "dv", OpenAITextCompletion("text-davinci-003", api_key, org_id)
)
kernel.add_text_embedding_generation_service(
    "ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id)
)

kernel.register_memory_store(memory_store=store)
kernel.import_skill(sk.core_skills.TextMemorySkill())

{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10d5946d0>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x12b032d50>}

## Adding Documents to Weaviate

Let's create some initial memories "About Me". We can add memories to our weaviate memory store by using `save_information_async`

In [5]:
COLLECTION = "AboutMe"

In [6]:
#Function for persisting a memory to Weaviate
import uuid

async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information_async(COLLECTION, 
                                               id = str(uuid.uuid4()), 
                                               text = 'When I turned 5 my parents gifted me goldfish for my birthday')
    
    await kernel.memory.save_information_async(COLLECTION,
                                              id = str(uuid.uuid4()),
                                              text = 'I love datascience')
    
    await kernel.memory.save_information_async(COLLECTION,
                                              id = str(uuid.uuid4()),
                                              text = 'I have a black nissan sentra')
    
    await kernel.memory.save_information_async(COLLECTION,
                                              id = str(uuid.uuid4()),
                                              text = 'my favourite food is popcorn')
    
    await kernel.memory.save_information_async(COLLECTION,
                                              id = str(uuid.uuid4()),
                                              text = 'I like to take long walks.')
    print("Sucessfully populated memories!")

In [7]:
await populate_memory(kernel)

Sucessfully populated memories!


In [8]:
#Conduct semantic search

result = await kernel.memory.search_async(COLLECTION, 'Do I have a pet?')
print(f"Retreived document: {result[0].text}")

Retreived document: When I turned 5 my parents gifted me goldfish for my birthday


You can retrieve `k` closest neighbours to a query as seen below:

In [9]:
result2 = await kernel.memory.search_async(COLLECTION, 'passion', limit=3)

for res in result2: print(f"{res.text} - Relevance: {res.relevance:.3f}")

I love datascience - Relevance: 0.894
my favourite food is popcorn - Relevance: 0.889
I like to take long walks. - Relevance: 0.879


We'll create a setup a prompt that will allow the LLM to use relevant context retreived from Weaviate to answer questions.

In [10]:
async def setup_RAG(kernel: sk.Kernel) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    You are a friendly and talkative AI.
    
    Answer to the user question: {{$user_input}} 
    
    You can, but don't have to, use relevant information provided here: {{$retreived_context}} 
    
    If you are not sure of the answer say "I am not sure."
    """.strip()

    rag_func = kernel.create_semantic_function(sk_prompt, max_tokens=200, temperature=0.8)

    context = kernel.create_new_context()

    #Need chat history now added to kernel context 
    context["chat_history"] = ""
    context["retreived_context"] = ""

    return rag_func, context

In [11]:
async def RAG(kernel: sk.Kernel, rag_func: sk.SKFunctionBase, context: sk.SKContext) -> bool:
    try:
        user_input = input("User:> ")
        context["user_input"] = user_input
    except KeyboardInterrupt:
        print("\n\nExiting chat...")
        return False
    except EOFError:
        print("\n\nExiting chat...")
        return False

    if user_input == "exit":
        print("\n\nExiting chat...")
        return False

    context["retreived_context"] = ''
    
    #Retrieve
    result = await kernel.memory.search_async(COLLECTION,context["user_input"], limit=5, min_relevance_score=0.5)
    
    for res in result:
        context["retreived_context"] += (res.text + '. \n')
    
    #Then generate
    answer = await kernel.run_async(rag_func, input_vars=context.variables)
    
    context["chat_history"] += f"\nUser:> {user_input}\nChatBot:> {answer}\n"

    print(f"\n\u001b[34mChatBot:> {answer}\u001b[0m \n\n\033[1;32m Source: {context['retreived_context']}\u001b[0m \n")
    return True

In [16]:
print("Setting up a RAG chat (with memory!)")
rag_func, context = await setup_RAG(kernel)

print("Begin chatting (type 'exit' to exit):\n")
chatting = True
while chatting:
    chatting = await RAG(kernel, rag_func, context)

Setting up a RAG chat (with memory!)
Begin chatting (type 'exit' to exit):

User:> What pets did I have when I was younger?

ChatBot:>  It appears that you were gifted goldfish for your 5th birthday. Do you remember if you had any other pets? 

 Source: When I turned 5 my parents gifted me goldfish for my birthday. 
I like to take long walks.. 
my favourite food is popcorn. 
I have a black nissan sentra. 
I love datascience. 
 

User:> What are some places I would enjoy traveling to in my car?

ChatBot:> 

There are plenty of great places you can enjoy traveling to in your car. Some ideas include road trips to national parks, scenic drives along the coast, or even taking a drive to explore a nearby city. Depending on your budget, you could also look for unique attractions like drive-in movie theaters, specialty shops, and restaurants. If you're interested in data science, consider taking a road trip to explore the tech scene in Silicon Valley or explore the birthplace of data science i

In [17]:
#The chathistory can be obtained from the context.
print(context.variables.get('chat_history')[1])


User:> What pets did I have when I was younger?
ChatBot:>  It appears that you were gifted goldfish for your 5th birthday. Do you remember if you had any other pets?

User:> What are some places I would enjoy traveling to in my car?
ChatBot:> 

There are plenty of great places you can enjoy traveling to in your car. Some ideas include road trips to national parks, scenic drives along the coast, or even taking a drive to explore a nearby city. Depending on your budget, you could also look for unique attractions like drive-in movie theaters, specialty shops, and restaurants. If you're interested in data science, consider taking a road trip to explore the tech scene in Silicon Valley or explore the birthplace of data science in New York. You can also stop for some popcorn along the way as a treat for yourself.

